# Mangrove Kandel Backtester - Multi-samples


## Engine


### 0. Imports


In [1]:
import math
import json
import numpy as np
import pandas as pd
from pandas import Series
from multiprocess import Pool
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from src.kandel_v2 import KandelConfig
from src.kandel_backtester import KandelBacktester, KandelBacktesterConfig

In [2]:
with open("config.json") as f:
    config = json.load(f)


window = config["window"]
initial_quote = config["initial_quote"]
initial_base = config["initial_base"]
vol_mult = config["vol_mult"]
n_points = config["n_points"]
step_size = config["step_size"]
vol_threshold = config["vol_threshold"]
vol_threshold_window = config["vol_threshold_window"]
asymmetric_exit_threshold = config["asymmetric_exit_threshold"]
samples_length = config["samples_length"]

del config

### 1. Preprocessing


#### Load data


In [3]:
prices_df = pd.read_csv("data/binance/ETHUSDC-2024-01>08_1s.csv", header=0, index_col=0)
prices_df.index = pd.to_datetime(prices_df.index, unit="s", utc=True)
prices_df["log_return"] = np.log(prices_df["price"] / prices_df["price"].shift(1))

In [4]:
prices_df["exit_vol"] = prices_df["log_return"].rolling(
    vol_threshold_window * 3600
).std() * np.sqrt(vol_threshold_window * 3600)
prices_df["exit_vol"] = prices_df["exit_vol"].fillna(0)

In [5]:
prices_df["window_vol"] = prices_df["log_return"].rolling(
    window * 3600
).std() * np.sqrt(window * 3600)
prices_df["window_vol"] = prices_df["window_vol"].fillna(0)

In [6]:
prices_df = prices_df.drop(columns=["log_return"])

#### Create samples


In [7]:
samples_length_s = (samples_length * 24 + window) * 3600

# Remove days used for volatility calculation
prices_df_trimmed = prices_df.iloc[(vol_threshold_window + window) * 3600 :]

prices_splitted = []
window_vol_splitted = []
exit_vol_splitted = []

for i in range(samples_length + math.ceil(window / 24)):
    prices_splitted += [
        prices_df_trimmed["price"][j : j + samples_length_s]
        for j in range(i * 3600 * 24, len(prices_df_trimmed["price"]), samples_length_s)
    ][:-1]
    window_vol_splitted += [
        prices_df_trimmed["window_vol"][j : j + samples_length_s]
        for j in range(i * 3600 * 24, len(prices_df_trimmed["window_vol"]), samples_length_s)
    ][:-1]
    exit_vol_splitted += [
        prices_df_trimmed["exit_vol"][j : j + samples_length_s]
        for j in range(i * 3600 * 24, len(prices_df_trimmed["exit_vol"]), samples_length_s)
    ][:-1]

del prices_df, prices_df_trimmed


### 2. Run backtester


In [8]:
def get_config(spot_price: float) -> KandelConfig:
    return KandelConfig(
        initial_quote=initial_quote,
        initial_base=initial_base / spot_price,
        window=window * 3600,
        vol_mult=vol_mult,
        n_points=n_points,
        step_size=step_size,
        vol_threshold=vol_threshold,
        vol_threshold_window=vol_threshold_window,
        asymmetric_exit_threshold=asymmetric_exit_threshold,
    )


def run_sample_backtest(
    prices: Series, window_vol: Series, exit_vol: Series
) -> tuple[float, float, float]:
    initial_spot = prices.iloc[0]
    config = get_config(initial_spot)
    kandel_backtester = KandelBacktester(
        prices=prices,
        window_vol=window_vol,
        exit_vol=exit_vol,
        backtester_config=KandelBacktesterConfig(
            position_history=False,
        ),
        kandel_config=config,
    )
    res, _, _ = kandel_backtester.run()
    final_spot = prices.iloc[-1]
    initial_mtm_quote = config["initial_quote"] + config["initial_base"] * initial_spot
    initial_mtm_base = initial_mtm_quote / initial_spot
    final_mtm_quote = res["quote"].iloc[-1] + res["base"].iloc[-1] * final_spot
    final_mtm_base = final_mtm_quote / final_spot

    price_diff = final_spot - initial_spot
    quote_returns = final_mtm_quote / initial_mtm_quote - 1
    base_returns = final_mtm_base / initial_mtm_base - 1

    return price_diff, quote_returns, base_returns

In [9]:
base_returns_arr = []
quote_returns_arr = []
final_price_diff_arr = []

with Pool(6) as pool:
    results = pool.starmap(run_sample_backtest, zip(prices_splitted, window_vol_splitted, exit_vol_splitted))
    for final_price_diff, quote_returns, base_returns, in results:
        final_price_diff_arr.append(final_price_diff)
        quote_returns_arr.append(quote_returns)
        base_returns_arr.append(base_returns)
    
   

100%|██████████| 345600/345600 [00:03<00:00, 93412.42it/s] 
/opt/anaconda3/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/opt/anaconda3/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/opt/anaconda3/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/opt/anaconda3/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/opt/anaconda3/l

## Analysis

In [10]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, specs=[[{}], [{}]], vertical_spacing=0.05, subplot_titles=("Over holding quote", "Hover holding base"))

quote_negative_returns = [[price, ret] for price, ret in zip(final_price_diff_arr, quote_returns_arr) if ret < 0]
quote_positive_returns = [[price, ret] for price, ret in zip(final_price_diff_arr, quote_returns_arr) if ret >= 0]

base_negative_returns = [[price, ret] for price, ret in zip(final_price_diff_arr, base_returns_arr) if ret < 0]
base_positive_returns = [[price, ret] for price, ret in zip(final_price_diff_arr, base_returns_arr) if ret >= 0]

fig.add_trace(
    go.Scatter(x=[row[0] for row in quote_negative_returns], y=[row[1] for row in quote_negative_returns], name="Returns", mode="markers", marker=dict(color="red")),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=[row[0] for row in quote_positive_returns], y=[row[1] for row in quote_positive_returns], name="Returns", mode="markers", marker=dict(color="green")),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=[row[0] for row in base_negative_returns], y=[row[1] for row in base_negative_returns], name="Returns", mode="markers", marker=dict(color="red")),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=[row[0] for row in base_positive_returns], y=[row[1] for row in base_positive_returns], name="Returns", mode="markers", marker=dict(color="green")),
    row=2, col=1
)


fig.update_xaxes(title_text="Price difference on period", row=2, col=1)
fig.update_yaxes(title_text="Returns", row=1, col=1)
fig.update_yaxes(title_text="Returns", row=2, col=1)

fig.update_layout(
    title="Returns vs Final Price Difference",
    showlegend=False,
    height=800,
    width=1200,
)

del quote_negative_returns, quote_positive_returns, base_negative_returns, base_positive_returns

fig.show()